In [28]:
print('ehllo')

ehllo


In [29]:
import boto3
boto3.__version__
import time

In [30]:
client = boto3.client('athena')

response1 = client.start_query_execution(
    QueryString='select  distinct state FROM "data-engineer"."claims"  ;',
    QueryExecutionContext={
        'Database': 'data-engineer'
       
    },
    ResultConfiguration={
        'OutputLocation':'s3://techkraft-dataengineer/athena-results/'
    }
)

time.sleep(2)
 

QueryExecutionId= response1['QueryExecutionId']
result = client.get_query_results(QueryExecutionId=QueryExecutionId)
print(result)




{'UpdateCount': 0, 'ResultSet': {'Rows': [{'Data': [{'VarCharValue': 'state'}]}, {'Data': [{'VarCharValue': 'AZ'}]}, {'Data': [{'VarCharValue': 'OR'}]}, {'Data': [{'VarCharValue': 'FL'}]}, {'Data': [{'VarCharValue': 'NY'}]}, {'Data': [{'VarCharValue': 'PA'}]}, {'Data': [{'VarCharValue': 'MO'}]}, {'Data': [{'VarCharValue': 'KY'}]}, {'Data': [{'VarCharValue': 'AL'}]}, {'Data': [{'VarCharValue': 'IA'}]}, {'Data': [{'VarCharValue': 'WY'}]}, {'Data': [{'VarCharValue': 'MI'}]}, {'Data': [{'VarCharValue': 'RI'}]}, {'Data': [{'VarCharValue': 'TN'}]}, {'Data': [{'VarCharValue': 'NE'}]}, {'Data': [{'VarCharValue': 'WA'}]}, {'Data': [{'VarCharValue': 'IN'}]}, {'Data': [{'VarCharValue': 'OH'}]}, {'Data': [{'VarCharValue': 'MD'}]}, {'Data': [{'VarCharValue': 'SC'}]}, {'Data': [{'VarCharValue': 'SD'}]}, {'Data': [{'VarCharValue': 'TX'}]}, {'Data': [{'VarCharValue': 'DC'}]}, {'Data': [{'VarCharValue': 'AR'}]}, {'Data': [{'VarCharValue': 'ID'}]}, {'Data': [{'VarCharValue': 'UT'}]}, {'Data': [{'VarChar

In [31]:
stateData=[]

In [32]:
for i in result['ResultSet']['Rows'][1:]:
    # print(i)
    state = i['Data'][0]['VarCharValue']
    stateData.append(state)
print(stateData)
    
    # print(value[0]['VarCharValue'])
    # city = value['VarCharValue']

['AZ', 'OR', 'FL', 'NY', 'PA', 'MO', 'KY', 'AL', 'IA', 'WY', 'MI', 'RI', 'TN', 'NE', 'WA', 'IN', 'OH', 'MD', 'SC', 'SD', 'TX', 'DC', 'AR', 'ID', 'UT', 'NH', 'ND', 'ME', 'WV', 'NJ', 'NM', 'CA', 'MA', 'OK', 'VA', 'WI', 'CO', 'LA', 'IL', 'CT', 'NC', 'GA', 'AK', 'MN', 'NV', 'KS', 'DE', 'HI', 'MS']


In [33]:
exe_id=[]

In [34]:
for z in stateData:
    response = client.start_query_execution(
    QueryString=f'select  \'{z}\' name, count(*) FROM "data-engineer"."claims" where state = \'{z}\' ;',
    QueryExecutionContext={
        'Database': 'data-engineer'
       
    },
    ResultConfiguration={
        'OutputLocation':'s3://techkraft-dataengineer/athena-results/'
    }
    )
    # time.sleep(2)
    id= response['QueryExecutionId']  
    exe_id.append(id)
    

In [35]:
# print(query_report['QueryExecution']['Status']['State'])
while True:
    state = client.get_query_execution(QueryExecutionId=id)['QueryExecution']['Status']['State']
    print(state)
    if(state == "SUCCEEDED"):
        break
    if(state == "FAILED" or state == "CANCELLED"):
        raise ValueError("Something went wrong with the query")
    # time.sleep(1)

RUNNING
SUCCEEDED


In [36]:
result = client.get_query_results(QueryExecutionId=id)
print(result)

{'UpdateCount': 0, 'ResultSet': {'Rows': [{'Data': [{'VarCharValue': 'name'}, {'VarCharValue': '_col1'}]}, {'Data': [{'VarCharValue': 'MS'}, {'VarCharValue': '4'}]}], 'ResultSetMetadata': {'ColumnInfo': [{'CatalogName': 'hive', 'SchemaName': '', 'TableName': '', 'Name': 'name', 'Label': 'name', 'Type': 'varchar', 'Precision': 2, 'Scale': 0, 'Nullable': 'UNKNOWN', 'CaseSensitive': True}, {'CatalogName': 'hive', 'SchemaName': '', 'TableName': '', 'Name': '_col1', 'Label': '_col1', 'Type': 'bigint', 'Precision': 19, 'Scale': 0, 'Nullable': 'UNKNOWN', 'CaseSensitive': False}]}}, 'ResponseMetadata': {'RequestId': 'a9f3b239-d539-46f4-a606-e13985910065', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Thu, 04 Aug 2022 11:11:34 GMT', 'x-amzn-requestid': 'a9f3b239-d539-46f4-a606-e13985910065', 'content-length': '938', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [37]:

keys=[]
for i in result['ResultSet']['Rows'][:1]:
    k=i['Data']
    for i in k:
        header=i['VarCharValue']
        keys.append(header)

data={}
for k in keys:
    data[k] = []

        

for i in exe_id:

    result = client.get_query_results(QueryExecutionId=i)
    for row in result['ResultSet']['Rows'][1:]:
        for i,k in enumerate(keys):
            print(i,k)
            data[k].append(row['Data'][i]['VarCharValue'])
        print(result['ResultSet']['Rows'])










0 name
1 _col1
[{'Data': [{'VarCharValue': 'name'}, {'VarCharValue': '_col1'}]}, {'Data': [{'VarCharValue': 'AZ'}, {'VarCharValue': '31'}]}]
0 name
1 _col1
[{'Data': [{'VarCharValue': 'name'}, {'VarCharValue': '_col1'}]}, {'Data': [{'VarCharValue': 'OR'}, {'VarCharValue': '11'}]}]
0 name
1 _col1
[{'Data': [{'VarCharValue': 'name'}, {'VarCharValue': '_col1'}]}, {'Data': [{'VarCharValue': 'FL'}, {'VarCharValue': '71'}]}]
0 name
1 _col1
[{'Data': [{'VarCharValue': 'name'}, {'VarCharValue': '_col1'}]}, {'Data': [{'VarCharValue': 'NY'}, {'VarCharValue': '69'}]}]
0 name
1 _col1
[{'Data': [{'VarCharValue': 'name'}, {'VarCharValue': '_col1'}]}, {'Data': [{'VarCharValue': 'PA'}, {'VarCharValue': '28'}]}]
0 name
1 _col1
[{'Data': [{'VarCharValue': 'name'}, {'VarCharValue': '_col1'}]}, {'Data': [{'VarCharValue': 'MO'}, {'VarCharValue': '18'}]}]
0 name
1 _col1
[{'Data': [{'VarCharValue': 'name'}, {'VarCharValue': '_col1'}]}, {'Data': [{'VarCharValue': 'KY'}, {'VarCharValue': '18'}]}]
0 name
1 _col

In [38]:
# print(result['ResultSet']['Rows'][:1])

    # for i,k in enumerate(key):
    #     print(i,k)
    # # print(i['Data'][0])

In [39]:
print(keys)

['name', '_col1']


In [40]:
print(data)

{'name': ['AZ', 'OR', 'FL', 'NY', 'PA', 'MO', 'KY', 'AL', 'IA', 'WY', 'MI', 'RI', 'TN', 'NE', 'WA', 'IN', 'OH', 'MD', 'SC', 'SD', 'TX', 'DC', 'AR', 'ID', 'UT', 'NH', 'ND', 'ME', 'WV', 'NJ', 'NM', 'CA', 'MA', 'OK', 'VA', 'WI', 'CO', 'LA', 'IL', 'CT', 'NC', 'GA', 'AK', 'MN', 'NV', 'KS', 'DE', 'HI', 'MS'], '_col1': ['31', '11', '71', '69', '28', '18', '18', '15', '6', '1', '22', '1', '19', '6', '19', '22', '29', '10', '9', '2', '127', '25', '6', '3', '4', '1', '2', '1', '9', '10', '4', '116', '15', '16', '35', '6', '30', '22', '23', '16', '25', '28', '1', '28', '13', '9', '7', '7', '4']}


In [41]:
import pandas as pd 
pd.__version__

'1.4.2'

In [42]:
df = pd.DataFrame(data)

In [43]:
df

,name,_col1
0,AZ,31
1,OR,11
2,FL,71
3,NY,69
4,PA,28
5,MO,18
6,KY,18
7,AL,15
8,IA,6
9,WY,1


In [45]:
df.to_csv('multileQuery_athena_python_output.csv',index=False)